In [1]:
import tensorflow as tf

In [0]:
from tensorflow.python.saved_model import utils as smutils

In [0]:
from tensorflow.python.saved_model import signature_constants
from tensorflow.python.saved_model import signature_def_utils
from tensorflow.python.saved_model import tag_constants

In [0]:
import logging

In [0]:
from pathlib import Path
meta_path = Path('/content/model.meta')
export_dir = Path('/content/')

In [0]:
input_tensor = tf.placeholder(dtype=tf.float32,
                              shape=(64,32,224,224), name='test_input')

In [0]:
input_dict = {'test_input':input_tensor}

In [25]:
input_dict

{'test_input': <tf.Tensor 'test_input:0' shape=(64, 32, 224, 224) dtype=float32>}

In [0]:
output_tensors = tf.placeholder(dtype=tf.float32, shape=(64,14), name='add_18')

In [0]:
output_dict = {'test_output':output_tensors}

In [23]:
output_dict

{'test_output': <tf.Tensor 'add_18:0' shape=(64, 14) dtype=float32>}

---- export for serving

In [0]:
export_dir = '/content/export' #some folder which is not existed or empty

In [0]:
log = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [0]:
!ls /content/export/variables

In [0]:
def export_for_serving(meta_path, export_dir, input_tensors, output_tensors):
    """
    Exports the Tensorflow .meta model to a frozen .pb Tensorflow serving
       format.
    :param meta_path: string, Path to the .meta TF proto file.
    :param export_dir: string, Path to directory where the serving model will
        be exported.
    :param input_tensor: dict, Input tensors dictionary of
        {name: TF placeholder} structure.
    :param output_tensors: dict, Output tensors dictionary of {name: TF tensor}
        structure.
    """
    g = tf.Graph()
    sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))
    graph_def = tf.GraphDef()

    with g.as_default():
        with open(meta_path, "rb") as f:
            graph_def.ParseFromString(f.read())

        # name argument must explicitly be set to an empty string, otherwise
        # TF will prepend an `import` scope name on all operations
        tf.import_graph_def(graph_def, name="")

        tensor_info_inputs = {name: smutils.build_tensor_info(in_tensor)
                              for name, in_tensor in input_tensors.items()}

        tensor_info_outputs = {name: smutils.build_tensor_info(out_tensor)
                               for name, out_tensor in output_tensors.items()}

        prediction_signature = signature_def_utils.build_signature_def(
            inputs=tensor_info_inputs,
            outputs=tensor_info_outputs,
            method_name=signature_constants.PREDICT_METHOD_NAME)

        builder = tf.saved_model.builder.SavedModelBuilder(export_dir)
        builder.add_meta_graph_and_variables(
            sess, [tag_constants.SERVING],
            signature_def_map={"predict_images": prediction_signature})
        builder.save()

        log.info("Input info:\n{}".format(tensor_info_inputs))
        log.info("Output info:\n{}".format(tensor_info_outputs))

In [49]:
export_for_serving(meta_path, export_dir, input_dict, output_dict)

INFO:tensorflow:No assets to save.


INFO:tensorflow:No assets to save.


INFO:tensorflow:No assets to write.


INFO:tensorflow:No assets to write.


INFO:tensorflow:SavedModel written to: /content/export/saved_model.pb


INFO:tensorflow:SavedModel written to: /content/export/saved_model.pb
INFO:__main__:Input info:
{'test_input': name: "test_input:0"
dtype: DT_FLOAT
tensor_shape {
  dim {
    size: 64
  }
  dim {
    size: 32
  }
  dim {
    size: 224
  }
  dim {
    size: 224
  }
}
}
INFO:__main__:Output info:
{'test_output': name: "add_18:0"
dtype: DT_FLOAT
tensor_shape {
  dim {
    size: 64
  }
  dim {
    size: 14
  }
}
}
